In [ ]:
from functools import partial
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from openretina.data_io.hoefling_2024.constants import FRAME_RATE_MODEL
from openretina.models.core_readout import CoreReadout, load_core_readout_from_remote
from openretina.insilico.stimulus_optimization.objective import ContrastiveNeuronObjective, SliceMeanReducer, IncreaseObjective
from openretina.insilico.stimulus_optimization.optimizer import optimize_stimulus
from openretina.insilico.stimulus_optimization.optimization_stopper import OptimizationStopper
from openretina.utils.plotting import play_stimulus, plot_stimulus_composition
from openretina.insilico.stimulus_optimization.regularizer import (ChangeNormJointlyClipRangeSeparately,
    TemporalGaussianLowPassFilterProcessor)

Load the pretrained neural network hoefling 2024 trained on low resolution videos.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = load_core_readout_from_remote("hoefling_2024_base_low_res", device=device)

Load the neural traces and the associated meta information based on the config of the model, and then extract the RGC group assignment for every neuron included in the model.

In [ ]:
# extract group assignments for each readout 
group_assignment = np.concatenate([model.data_info["sessions_kwargs"][k]["group_assignment"] for k in model.readout.readout_keys()])
type_to_idc = {int(t): np.where(group_assignment == t)[0].tolist() for t in set(group_assignment)}

Define a function that optimizes a stimulus based on an objective and then plots this optimized stimulus.

In [ ]:
def optimize_and_plot_stimulus(stimulus_shape, objective) -> None:
    torch.manual_seed(40)
    stimulus = torch.randn(stimulus_shape, requires_grad=True, device=device)
    stimulus_clipper = ChangeNormJointlyClipRangeSeparately(
        min_max_values=[(-0.6, 6.2), (-0.9, 6.2)],
        norm=30.0,
    )
    stimulus_lowpass_filter = TemporalGaussianLowPassFilterProcessor(sigma=0.5, kernel_size=5, device=device)
    stimulus_postprocessor_list = [stimulus_clipper, stimulus_lowpass_filter]

    # Clip the initial stimulus to the expected range
    stimulus.data = stimulus_clipper.process(stimulus.data * 0.1)

    optimize_stimulus(
        stimulus,
        optimizer_init_fn=partial(torch.optim.SGD, lr=100.0),
        objective_object=objective,
        optimization_stopper=OptimizationStopper(max_iterations=10),
        stimulus_postprocessor=stimulus_postprocessor_list,
    )

    fig_axes_tuple = plt.subplots(2, 2, figsize=(7 * 3, 12))
    axes: np.ndarray = fig_axes_tuple[1]  # type: ignore

    stim_length = stimulus_shape[2]
    plot_stimulus_composition(
        stimulus=stimulus[0].detach().cpu().numpy(),
        temporal_trace_ax=axes[0, 0],
        freq_ax=axes[0, 1],
        spatial_ax=axes[1, 0],
        highlight_x_list=[(stim_length-10,stim_length-1)],
    )
    return stimulus


Define two objective functions:
- objective_mei optimizes an most exciting stimulus such that cells of type 28 activate strongly.
- objective_mds optimizes a most discriminatory stimulus such that cells of type 28 activate strongly but minimizes the response of other on and off cells.

In [ ]:
stimulus_shape = model.stimulus_shape(time_steps=40)

reducer = SliceMeanReducer(axis=0, start=10, length=10)
objective_mei = IncreaseObjective(
    model,
    neuron_indices=type_to_idc[28],
    data_key=None,
    response_reducer=reducer,
)

off_cell_idc = sum([type_to_idc[i] for i in range(1, 10)], [])
on_off_cell_idc = sum([type_to_idc[i] for i in range(10, 15)], [])
on_cell_idc = sum([type_to_idc[i] for i in range(15, 28)], [])
objective_mds = ContrastiveNeuronObjective(
    model, 
    on_cluster_idc=type_to_idc[28],
    off_cluster_idc_list=[on_cell_idc, off_cell_idc],
    data_key=None,
    response_reducer=reducer,
    temperature=1.6,
)

In [ ]:
stimulus_mei = optimize_and_plot_stimulus(stimulus_shape, objective_mei)

In [ ]:
stimulus_mds = optimize_and_plot_stimulus(stimulus_shape, objective_mds)

We now use these stimuli and simulate the responses of the neural network.

In [ ]:
responses_mds = model.forward(stimulus_mds)
responses_mds_np = responses_mds[0].detach().transpose(1, 0).cpu().numpy()
responses_mei = model.forward(stimulus_mei)
responses_mei_np = responses_mei[0].detach().transpose(1, 0).cpu().numpy()

For the MEI we see that the average response to neurons of group 28 is high, but the response of the other on cells is even higher.
In contrast, the average response of the on cells to the MDS stimulus is lower compared to the response of cells of group 28 cells.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3), sharey=True)

offset = 20
time = np.arange(offset, offset + responses_mds_np[0].shape[0]) / FRAME_RATE_MODEL
for resp, linestyle in [(responses_mei_np, "--"), (responses_mds_np, "-")]:
    l1, = ax.plot(time, resp[type_to_idc[28]].mean(axis=0), label="Group 28", color="red", linestyle=linestyle)
    l2, = ax.plot(time, resp[on_cell_idc].mean(axis=0), label="On Cells", color="green", linestyle=linestyle)
    l3, = ax.plot(time, resp[off_cell_idc].mean(axis=0), label="Off Cells", color="purple", linestyle=linestyle)

ax.set_ylabel("Average Responses [AU]")
ax.set_xlabel("Time [s]")
ax.fill_betweenx(
    ax.get_ylim(), 
    (offset + reducer.start) / FRAME_RATE_MODEL, 
    (offset + reducer.start + reducer.length) / FRAME_RATE_MODEL, 
    color="k", alpha=0.1
)
ax.legend(handles=[l1, l2, l3], loc="upper center", bbox_to_anchor=(0.4, 1.00))
sns.despine()